In [1]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.formula.api as smf

In [2]:
%load_ext watermark
az.style.use("arviz-darkgrid")
np.random.seed(1211)

#### Code 7.1

In [3]:
sppnames = ["afarensis", "africanus", "habilis", "boisei", "rudolfensis", "ergaster", "sapiens"]
brainvolcc = [438, 452, 612, 521, 752, 871, 1350]
masskg =  [37.0, 35.5, 34.5, 41.5, 55.5, 61.0, 53.5]

d = pd.DataFrame.from_dict({"species": sppnames, "brain": brainvolcc, "mass": masskg})
d

,species,brain,mass
0,afarensis,438,37.0
1,africanus,452,35.5
2,habilis,612,34.5
3,boisei,521,41.5
4,rudolfensis,752,55.5
5,ergaster,871,61.0
6,sapiens,1350,53.5


#### Code 7.2

In [4]:
d["mass_std"] = (d["mass"] - d["mass"].mean()) / d["mass"].std()
d["brain_std"] = d["brain"] / d["brain"].max()

#### Code 7.3

In [5]:
priors = {
    "Intercept": bmb.Prior("Normal", mu=0.5, sd=1),
    "mass_std": bmb.Prior("Normal", mu=0, sd=10),
    "sigma": bmb.Prior("Exponential", lam=1),
}

model_7_1 = bmb.Model("brain_std ~ mass_std", d, priors=priors)
results_7_1 = model_7_1.fit()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [brain_std_sigma, mass_std, Intercept]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 3 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


#### Code 7.4

In [6]:
model_7_1_OLS = smf.ols("brain_std ~ mass_std", data=d).fit()
mean = model_7_1_OLS.params.values
cov = np.diag(model_7_1_OLS.cov_params())
post = stats.multivariate_normal.rvs(mean=mean, cov=cov, size=10000)

#### Code 7.5

In [7]:
def var2(x):
    return np.mean(x ** 2) - np.mean(x) ** 2

In [8]:
model_7_1.posterior_predictive(results_7_1, 1000)

/home/tomas/anaconda3/envs/bmbrsrc/lib/python3.9/site-packages/pymc3/sampling.py:1689: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  warnings.warn(


In [9]:
r = results_7_1.posterior_predictive["brain_std"].values.reshape(-1, 7).mean(axis=0) - d["brain_std"].values
resid_var = var2(r)
outcome_var = var2(d["brain_std"])
1 - resid_var / outcome_var

0.5062199789556512

#### Code 7.6

In [10]:
def R2_is_bad(model, result):
    model.posterior_predictive(result, 1000)
    r = result.posterior_predictive["brain_std"].values.reshape(-1, 7).mean(axis=0) - d["brain_std"].values
    return 1 - var2(r) / var2(d["brain_std"])

#### Code 7.7

Here we use the `common` key in the priors dictionary. This means that all the common predictors (fixed effects) are going to receive the same prior, a normal with mean 0 and sd of 10, as our inspection below confirms. We also start with a baseline formula, and add terms to the formula as we create models.

In [11]:
formula = "brain_std ~ mass_std + I(mass_std ** 2)"
priors = {
    "Intercept": bmb.Prior("Normal", mu=0.5, sd=1),
    "common": bmb.Prior("Normal", mu=0, sd=10),
    "sigma": bmb.Prior("Exponential", lam=1),
}

In [12]:
model_7_2 = bmb.Model(formula, d, priors=priors)
model_7_2

Formula: brain_std ~ mass_std + I(mass_std ** 2)
Family name: Gaussian
Link: identity
Observations: 7
Priors:
  Intercept ~ Normal(mu: 0.5, sd: 1)
  mass_std ~ Normal(mu: 0, sd: 10)
  I(mass_std ** 2) ~ Normal(mu: 0, sd: 10)
  sigma ~ Exponential(lam: 1)

In [15]:
results_7_2 = model_7_2.fit(tune=4000, target_accept=0.99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [brain_std_sigma, I(mass_std ** 2), mass_std, Intercept]


Sampling 2 chains for 4_000 tune and 1_000 draw iterations (8_000 + 2_000 draws total) took 19 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [14]:
formula += " + I(mass_std ** 3)"
model_7_3 = bmb.Model(formula, d, priors=priors)
formula += " + I(mass_std ** 4)"
model_7_4 = bmb.Model(formula, d, priors=priors)
formula += " + I(mass_std ** 5)"
model_7_5 = bmb.Model(formula, d, priors=priors)
formula += " + I(mass_std ** 6)"
model_7_6 = bmb.Model(formula, d, priors=priors)

In [16]:
results_7_3 = model_7_3.fit(tune=4000, target_accept=0.99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [brain_std_sigma, I(mass_std ** 3), I(mass_std ** 2), mass_std, Intercept]


Sampling 2 chains for 4_000 tune and 1_000 draw iterations (8_000 + 2_000 draws total) took 30 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [17]:
results_7_4 = model_7_4.fit(tune=4000, target_accept=0.99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [brain_std_sigma, I(mass_std ** 4), I(mass_std ** 3), I(mass_std ** 2), mass_std, Intercept]


Sampling 2 chains for 4_000 tune and 1_000 draw iterations (8_000 + 2_000 draws total) took 96 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [18]:
results_7_5 = model_7_5.fit(tune=4000, target_accept=0.99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [brain_std_sigma, I(mass_std ** 5), I(mass_std ** 4), I(mass_std ** 3), I(mass_std ** 2), mass_std, Intercept]


Sampling 2 chains for 4_000 tune and 1_000 draw iterations (8_000 + 2_000 draws total) took 324 seconds.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.


In [20]:
results_7_6 = model_7_6.fit(tune=4000, target_accept=0.99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [brain_std_sigma, I(mass_std ** 6), I(mass_std ** 5), I(mass_std ** 4), I(mass_std ** 3), I(mass_std ** 2), mass_std, Intercept]


Sampling 2 chains for 4_000 tune and 1_000 draw iterations (8_000 + 2_000 draws total) took 569 seconds.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 22 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


/home/tomas/anaconda3/envs/bmbrsrc/lib/python3.9/site-packages/pymc3/sampling.py:1689: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  warnings.warn(


0.9340830170940373

In [22]:
R2_is_bad(model_7_2, results_7_2)

/home/tomas/anaconda3/envs/bmbrsrc/lib/python3.9/site-packages/pymc3/sampling.py:1689: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  warnings.warn(


0.5675915096895109

In [23]:
R2_is_bad(model_7_3, results_7_3)

/home/tomas/anaconda3/envs/bmbrsrc/lib/python3.9/site-packages/pymc3/sampling.py:1689: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  warnings.warn(


0.6771137110950032

In [24]:
R2_is_bad(model_7_4, results_7_4)

/home/tomas/anaconda3/envs/bmbrsrc/lib/python3.9/site-packages/pymc3/sampling.py:1689: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  warnings.warn(


0.8142991535749295

In [25]:
R2_is_bad(model_7_5, results_7_5)

/home/tomas/anaconda3/envs/bmbrsrc/lib/python3.9/site-packages/pymc3/sampling.py:1689: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  warnings.warn(


0.9787767930364683

In [26]:
R2_is_bad(model_7_6, results_7_6)

0.9103729301687322